In [44]:
import random, re, csv
import numpy as np
import MLData
#for features and labels
from sklearn.model_selection import train_test_split
#for analysis methods
from sklearn.cluster import KMeans
from sklearn import svm
#for evaluation
from scipy import stats
from sklearn.metrics import accuracy_score
from sklearn import metrics
from sklearn.model_selection import cross_val_score

#=========================
# Get Data for analysis
#=========================
#get raw data from dataset
ilpddatafiledir = '/afs/inf.ed.ac.uk/user/s14/s1458248/workspace/vinlccanalysissystem/src/main/resources/ilpddata/ilpddata2.csv'
ilpddata =MLData.get_ilpddata(ilpddatafiledir)
interdata = MLData.get_interdata_sim(ilpddata)
mixdata = MLData.get_mixdata(ilpddata, interdata)

'''
# get features and labels from raw data
mldata = MLData.get_features_labels(mixdata,13,0.4)
train_features=mldata['train_features']
train_labels=mldata['train_labels']
test_features=mldata['test_features']
test_labels=mldata['test_labels']
'''

"\n# get features and labels from raw data\nmldata = MLData.get_features_labels(mixdata,13,0.4)\ntrain_features=mldata['train_features']\ntrain_labels=mldata['train_labels']\ntest_features=mldata['test_features']\ntest_labels=mldata['test_labels']\n"

In [45]:
#=======================================
# Interaction data analysis
#=======================================
#special variable
num_clusters=2

#build interdata features
interdata_features=np.array([np.zeros(len(interdata[0])-1)])
num_interdata_features=len(interdata[0])-1

for i in interdata:
    interdata_features=np.append(interdata_features,[i[0:num_interdata_features]],axis=0)
interdata_features=np.delete(interdata_features,0,0)

#do clustering on interdata features(kmeans)
kmeans = KMeans(n_clusters=num_clusters, random_state=0).fit(interdata_features)
interdata_km_labels=kmeans.labels_
#kresult=kmeans.predict(interdata_sim_features)

#build divided mixdata
mixdata_div=[]
for i in range(0,len(interdiv_labels)):
    mixdata_div=mixdata_div+[mixdata[i]+[interdata_km_labels[i]]]

    
'''
mixdata_div=[]
for i in range(1,num_clusters+1):
    mixdata_div=mixdata_div+[[]]
for i in range(0,len(interdiv_labels)):
        mixdata_div[interdiv_labels[i]]=mixdata_div[interdiv_labels[i]]+[mixdata[i]]
'''
'''      
print sampleset_0.shape[0]+sampleset_1.shape[0]
print stats.ttest_ind(kresult,inter_test_labels)
print accuracy_score(kresult,inter_test_labels)

print metrics.adjusted_rand_score(kresult,inter_test_labels)
'''

'      \nprint sampleset_0.shape[0]+sampleset_1.shape[0]\nprint stats.ttest_ind(kresult,inter_test_labels)\nprint accuracy_score(kresult,inter_test_labels)\n\nprint metrics.adjusted_rand_score(kresult,inter_test_labels)\n'

In [48]:
#==========================================================
# sbatract bio information patterns from divided samples
#==========================================================

ml_mixdata_div = MLData.get_features_labels(mixdata_div,14,0.4)
train_features=ml_mixdata_div['train_features']
train_labels=ml_mixdata_div['train_labels']
test_features=ml_mixdata_div['test_features']
test_labels=ml_mixdata_div['test_labels']


svmclf=svm.SVC(C=1)
svmclf.fit(train_features,train_labels)
svmclfresult=svmclf.predict(test_features)

scores=cross_val_score(svmclf,train_features,train_labels,cv=4)


print scores
print accuracy_score(test_labels,svmclfresult)


[ 0.71590909  0.72413793  0.72093023  0.72093023]
0.711206896552
